In [1]:
!pip install -q git+https://github.com/openai/CLIP.git
import torch
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from torchvision.models import detection
import torchvision
from tqdm import tqdm, tqdm_notebook
from PIL import Image

     |████████████████████████████████| 64 kB 1.2 MB/s 


In [ ]:
!unzip drive/MyDrive/leadersd/cat_dog_test.zip

In [5]:
import torch
import clip
from PIL import Image

device =  "cuda"
model, preprocess = clip.load("ViT-B/32", device=device)
classes = ["a dog", "a cat", "a bird"] 

print(len(classes))

text = clip.tokenize(classes).to(device)

3


In [9]:
path_parent = "cat_dog_test/"
for dirname in os.listdir(path_parent):

  path = os.path.join(path_parent, dirname)
  print(dirname, len(os.listdir(path)))

a dog 1499
a cat 1499


In [16]:
count = 0
res = {}
path_parent = "cat_dog_test"
for dirname in os.listdir(path_parent):

  path = os.path.join(path_parent, dirname)
  count = 0

  for filename in tqdm(os.listdir(path)[:200]):

    image = torch.tensor(np.asarray(Image.open(os.path.join(path, filename)))).permute(2, 0, 1)
    image = torchvision.transforms.Resize((80, 80))(image)
    image = torchvision.transforms.ToPILImage()(image)

    image = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
      logits_per_image, logits_per_text = model(image, text)
      probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    
    if (classes[np.argmax(probs)] == dirname):
      count += 1

  print("\n", dirname, count / len(os.listdir(path)[:200]))
  res[dirname] = count / len(os.listdir(path)[:200])

100%|██████████| 200/200 [00:16<00:00, 12.00it/s]



 a dog 0.99


100%|██████████| 200/200 [00:16<00:00, 12.12it/s]


 a cat 0.97


In [19]:
print(res)

{'a dog': 0.99, 'a cat': 0.97}


In [18]:
all = 0
tr = 0
for breed in res.keys():
  tr += len(os.listdir(os.path.join(path_parent, breed))[:200]) * res[breed]
  all += len(os.listdir(os.path.join(path_parent, breed))[:200])
print(tr/all)

0.98
